In [ ]:
# sc.list_packages()
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("boto3")
sc.install_pypi_package("rapidfuzz")

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1668471767937_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/01/21/d71d43fab3c19e30bac8468c01e8fd03e7649f98f068c1e900f293142d5e/boto3-1.26.9-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/c6/af903b5fab3f9b5b1e883f49a770066314c6dcceb589cf938d48c89556c1/s3transfer-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/26/19/8657d085b575f594f96d3b6b0c6113e4bfd30ec7265486ababb5d34cd498/botocore-1.29.9-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6f/de/5be2e3eed8426f871b170663333a0f627fc2924cc386cd41be065e7ea870/urllib3-1.26.12-py2.py3-none-any.whl


You are using pip version 9.0.1, however 

In [2]:
confs = ["spark.yarn.executor.memoryOverhead","spark.sql.shuffle.partitions","spark.network.timeout"]
print(sc._conf.get('spark.driver.memory'))
print(sc._conf.get('spark.app.name'))
print(sc._conf.get('spark.submit.deployMode'))
print(sc._conf.get("spark.yarn.executor.memoryOverhead"))
print(sc._conf.get("spark.sql.files.maxPartitionBytes"))
for conf in confs:
    print(conf,":",sc._conf.get(conf))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2048M
livy-session-1
client
None
None
spark.yarn.executor.memoryOverhead : None
spark.sql.shuffle.partitions : None
spark.network.timeout : None

# Imports

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth
import boto3
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, MapType, IntegerType, DoubleType
from pyspark.sql.types import *
from pyspark.sql import functions as F
from collections import Counter
import pyspark
import rapidfuzz
from rapidfuzz import fuzz


# import s3fs
# import boto3
#How to pip install while creation of cluster

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:

#          .config("spark.memory.offHeap.enabled","true") \
#          .config("spark.memory.offHeap.size","10g") \
# .config("spark.submit.deployMode","client") \
#         .config("spark.driver.memory","200g") \
#         .config("spark.executor.memory","50g") \
#         .config("spark.emr.default.executor.memory", '50g') \

# spark.conf.set("spark.sql.shuffle.partitions",100)

#Number of partitions should be equal to or greater than the cores to achieve parallelism
# config = pyspark.SparkConf().setAll([('spark.executor.memory', '80'), ('spark.executor.cores', '12'), \
#                                      ('spark.cores.max', '240'), ('spark.driver.memory','300g'), ('spark.sql.shuffle.partitions','80')])
# spark.sparkContext.stop()
config = pyspark.SparkConf().setAll([('spark.network.timeout', '100800s')]) #, ('spark.executor.cores', '12')])
spark = SparkSession.builder.config(conf=config).appName('ProcessingData') \
        .getOrCreate()
# spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext.getOrCreate(conf=config)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Task
Read data of specific countries. Within them, if any org in any row is a close match to the list of primary or secondary firms then return a one. Groupby with average tone, and count for total, primary, and secondary firms. Groupby level, country, month.

Countries:
1. Senegal - SG
2. Malawi - MI
3. Mozambique - MZ

Primary Firms:\
Cadila \
Cipla \
Dr. Reddy's \
Ipca \
Lupin / pharma dynamics \ 
Maiden \
Orchid \
Panacea Biotech \
Ranbaxy \
Sun 

Secondary Firms:\
Actiza \
Ajanta Pharma \
Atul \
Aurobindo \
Bal Pharma \
Glenmark \
Global Pharma \
InterMed Pharma \
JoinHub Pharma \
KEC Ltd. \
Kirloskar Bros \
Maan Pharma \
MS Pharma \
Promac \
Pyramid Pharma \
SenegIndia \
XL Laboratories \ 
Weefsel Pharma


## 1.  Reading Files

In [5]:
from boto3 import client

def return_yearly_parquet_files(year):
    print(f"Return file list for the year: {year}")
    file_list = []
    conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
    for key in conn.list_objects(Bucket='kcsra', Prefix = f"{str(year)} GDELT GKG PG")['Contents']:
        file = "s3://kcsra/"+ key['Key']
        if "parquet" in file:
            file_list.append(file)
    return file_list

def return_correct_file_paths(year_file_list):
    print("Returning the working parquet files ...")
    correct_file_paths = []
    for i,file in enumerate(year_file_list):

        try:
            df = spark.read.parquet(file)
            correct_file_paths.append(file)
        except Exception as e:
            print(f"[{str(i+1)}/{len(year_file_list)}]")
            print("->ERROR while reading:")
            print(file)
            print()
    return correct_file_paths

def return_df(correct_file_paths, req_cols, grid_id = None ):
    print("Constructing a dataframe from required file paths")

    df = spark.read.parquet(*correct_file_paths).select(*req_cols).dropDuplicates(['gkgrecordid','GRID_IDs'])
    
    if grid_id:
        print("Filter Grid ID:", grid_id)
        df = df.filter(df.GRID_IDs == grid_id)

    df = df.withColumn('day',dayofmonth(df.date)).withColumn('month',month(df.date))
    print("Total rows:",df.count())
    # df.printSchema()
    return df

def extract_tone(tone_string):
    try:
        tone_float = float(re.findall("tone=(.*?),",tone_string)[0])
        return tone_float
    except Exception as e:
        print("Error extracting tone:" + str(exception))
        print(tone_string)
        print("="*20)
        return None
    
def save_tone_in_new_col(df):
    print("Saving the tone in a new column")
    extract_tone_udf = udf(extract_tone, DoubleType())
    df = df.withColumn('Extracted_tone', extract_tone_udf(df['tone'])).drop("tone").withColumnRenamed("Extracted_tone", "tone")
    return df

@udf(returnType=ArrayType(StringType()))
def retrieve_theme_list(themes):
    if not themes:
        return {}
    themes = str(themes)
    return list(Counter(re.findall(f"theme=(.*?),",themes)).keys())

@udf(returnType=ArrayType(StringType())) 
def retrieve_organization_list(organizations):
    if not organizations:
        return {}
    organizations = str(organizations)
    return list(Counter(re.findall(f"organization=(.*?),",organizations)).keys())

@udf(returnType=StringType())
def aggregate_category(category):
    super_counter = Counter({})
    for row in category:
        super_counter += Counter(row)
    return ", ".join([f"{categ[0]}:{categ[1]}" for categ in super_counter.most_common(20)])


def are_strings_similar(target_str, data_str, match_threshold):
    target_str, data_str = str(target_str).lower(), str(data_str).lower()
    partial_match, sorted_match = fuzz.partial_ratio(target_str, data_str), fuzz.token_sort_ratio(target_str, data_str)

    if max(partial_match, sorted_match) >= match_threshold:
        # print(max(partial_match, sorted_match))
        return True
    else:
        return False


def compare_string_matches_lists(target_list, data_list, match_threshold = 75):
    for target_str in target_list:
        for data_str in data_list:
            if are_strings_similar(target_str, data_str, match_threshold):
                # print("Match Found", f"| {target_str} : {data_str}")
                return 1
    return 0


def udf_compare_strings_lists(target_list, match_threshold):
    return udf(lambda data_list: compare_string_matches_lists(target_list, data_list, match_threshold))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
target_str = "Promac"
data_str = "promac llc"
match_threshold = 78
are_strings_similar(target_str, data_str, match_threshold)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

# MAIN

In [7]:
#Collect all the orgs in a list and run the matcher function to label primary and secondary firms

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
primary_firms = """Cadila 
Cipla 
Dr. Reddy's 
Ipca Lab
Lupin 
pharma dynamics 
Maiden 
Orchid 
Panacea Biotech 
Ranbaxy 
Sun Pharma"""

secondary_firms = """Actiza 
Ajanta Pharma 
Atul 
Aurobindo 
Bal Pharma 
Glenmark 
Global Pharma 
InterMed Pharma 
JoinHub Pharma 
KEC Ltd. 
Kirloskar Bros 
Maan Pharma 
MS Pharma 
Promac 
Pyramid Pharma 
SenegIndia 
XL Laboratories 
Weefsel Pharma"""

prim_firm_list = [firm.strip() for firm in primary_firms.split("\n")]
sec_firm_list = [firm.strip() for firm in secondary_firms.split("\n")]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# match_str = """[Google, Ballistic Research Centre, Institute Of Forensic Sciences, Torrent Pharmaceuticals, Bajaj Alliance, Directorate Of Forensic Sciences, Institute Of Behavioral Sciences, Gujarat Forensic Sciences University, Formula Group, Institute Of Research Development, Congress Mp Shashi Tharoor, Telecom Day, Us Central Intelligence Agency, Cadila Pharmaceuticals, Directorate Of Forensic Science Laboratory]"""

# data_list = [x.strip() for x in match_str.strip().strip("[").strip("]").split(",")]
# compare_string_matches_lists(prim_firm_list, data_list, match_threshold = 75)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
countries = ["SG"]#["MZ", "MI", "SG"]
match_threshold = 91
req_cols = ['gkgrecordid','date','tone','country', 'year','GRID_IDs', 'organizations']
for country in countries:
    for year in range(2018, 2019):
        print(f"Country: {country} | Year: {year}")
        file_list = return_yearly_parquet_files(year)
        file_list2 = [file for file in file_list if country in file]
        correct_file_paths = return_correct_file_paths(file_list2)
        df = return_df(correct_file_paths, req_cols)
        df = save_tone_in_new_col(df)
        df = df.dropna(subset = ["tone"])

        req_cols_grpby = ['country', 'day', 'month', 'year', 'tone', 'PrimaryFirms', 'SecondaryFirms']
        df2 = df.withColumn("Org_list", retrieve_organization_list(F.col("organizations")))
        df.unpersist()
        df3 = df2.withColumn('PrimaryFirms', udf_compare_strings_lists(prim_firm_list, match_threshold)(F.col("Org_list")).cast(IntegerType()))\
            .withColumn('SecondaryFirms', udf_compare_strings_lists(sec_firm_list, match_threshold)(F.col("Org_list")).cast(IntegerType()))\
            .select(*req_cols_grpby)
        df2.unpersist()
        df4 = df3.withColumn("PrimaryFirmTone", F.col("PrimaryFirms") * F.col("tone")).withColumn("SecondaryFirmTone", F.col("SecondaryFirms") * F.col("tone"))
        df3.unpersist()

        aggregation_cols = ['country','month','year']
        df5 = df4.groupby(*aggregation_cols).agg(F.count(F.col("tone")).alias('TotalArticles'),\
                                   F.avg('tone').alias('AverageSentiment'),\
                                  F.sum('PrimaryFirms').alias('PrimaryFirms_TotalArticles'),\
                                   F.sum('PrimaryFirmTone').alias('SumPrimaryFirmTone'),\
                                  F.sum('SecondaryFirms').alias('SecondaryFirms_TotalArticles'),\
                                  F.sum('SecondaryFirmTone').alias('SumSecondaryFirmTone'))
        df4.unpersist()
        df6 = df5.withColumn("AverageSentiment_PrimaryFirms", F.col("SumPrimaryFirmTone")/F.col("PrimaryFirms_TotalArticles"))\
            .withColumn("AverageSentiment_SecondaryFirms", F.col("SumSecondaryFirmTone")/F.col("SecondaryFirms_TotalArticles"))\
            .drop("SumPrimaryFirmTone","SumSecondaryFirmTone").fillna(0)
        df5.unpersist()
        
        df6.repartition(1).write.option("header",True) \
           .csv(f"s3://shrivats-dev/Tasks/18_Primary_Secondary_Firms_Sentiment/{country}/Monthly/{year}")     
        print("="*50)
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Country: SG | Year: 2018
Return file list for the year: 2018
Returning the working parquet files ...
Constructing a dataframe from required file paths
Total rows: 374941
Saving the tone in a new column
DataFrame[gkgrecordid: string, date: timestamp, country: string, year: string, GRID_IDs: bigint, organizations: string, day: int, month: int, tone: double]
DataFrame[gkgrecordid: string, date: timestamp, country: string, year: string, GRID_IDs: bigint, organizations: string, day: int, month: int, tone: double, Org_list: array<string>]
DataFrame[country: string, day: int, month: int, year: string, tone: double, PrimaryFirms: int, SecondaryFirms: int]
DataFrame[country: string, day: int, month: int, year: string, tone: double, PrimaryFirms: int, SecondaryFirms: int, PrimaryFirmTone: double, SecondaryFirmTone: double]
DataFrame[country: string, month: int, year: string, TotalArticles: bigint, AverageSentiment: double, PrimaryFirms_TotalArticles: bigint, SumPrimaryFirmTone: double, Secondary

## Joining Files

In [13]:
countries = ["MZ", "MI", "SG"]
for country in countries:
    df_combined = spark.read.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/18_Primary_Secondary_Firms_Sentiment/{country}/Monthly/*/*.csv")
    df_combined.repartition(1).write.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/18_Primary_Secondary_Firms_Sentiment/{country}/Monthly/AllYears")    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_combined_all_countries = spark.read.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/18_Primary_Secondary_Firms_Sentiment/*/Monthly/2*/*.csv")
df_combined_all_countries = df_combined_all_countries.withColumn("month", F.col("month").cast(IntegerType()))
df_combined_all_countries_sorted = df_combined_all_countries.sort("country","year","month")
df_combined_all_countries_sorted.repartition(1).write.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/18_Primary_Secondary_Firms_Sentiment/MZ_MI_SG/Monthly")  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…